<a href="https://colab.research.google.com/github/Sandr001/Data-Mining-MGT7216/blob/development/MGT7216_Sentiment_Analytics_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis using Semi-Supervised Learning - 1**
**Self Learning Approach:**

**Import necessary packages**

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [91]:
!pip install langdetect
!pip install googletrans==4.0.0-rc1

from langdetect import detect
from googletrans import Translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=b184810c3de715f0f63d56bb226e361118821565850fadef9d859ff3fbd0fee8
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 

In [88]:
df = pd.read_excel("/content/drive/MyDrive/Data Mining/A_II_Emotion_Data_Student_Copy_Final.xlsx")

**Descriptive Statistics**

In [89]:
print("\033[1mThe dimension of the data:\033[0m", df.shape)

null_counts = df.isnull().sum()
print("\n\033[1mColumn wise Null value counts:\033[0m")
print(null_counts)

for column in ["brand_name_", "country_", "star_rating_"]:
    print("\n\033[1mUnique levels for column '{}':\033[0m".format(column), df[column].unique())

print(f"\n\033[1mCount for the Countries:\033[0m {len(df['country_'].unique())}")

print(f"\n\033[1m Labeled Data Count:\033[0m {df['emotions_'].notnull().sum()}")

print(f"\n\033[1m Unlabeled Data Count:\033[0m {df['emotions_'].isnull().sum()}")

The dimension of the data: (5722, 6)

Column wise Null value counts:
ID_                 0
brand_name_         0
country_            0
star_rating_        0
emotions_        5095
text_reviews_       0
dtype: int64

Unique levels for column 'brand_name_': ['Z_' 'H_']

Unique levels for column 'country_': ['US' 'GB' 'FRI' 'HR' 'NO' 'IE' 'CA' 'DK' 'PT' 'ES' 'AT' 'IN' 'IT' 'NL'
 'DE' 'AU' 'CZ' 'RO' 'FI' 'FR' 'GR' 'SI' 'HK' 'HU' 'MY' 'UA' 'MX' 'TR'
 'ZA' 'BE' 'RS' 'AE' 'SE' 'EN' 'CH' 'MAL' 'PL' 'LV' 'MK' 'IS' 'LB' 'PH'
 'JP' 'SG' 'LU' 'PE' 'SK' 'LT' 'BR' 'CO' 'TN' 'TH' 'CY' 'IL' 'HN' 'DO'
 'VN' 'CL' 'AR' 'UY' 'IR' 'PK' 'PA' 'NZ' 'GE' 'RE' 'MT' 'VE' 'CW' 'NI'
 'PR' 'AM' 'CN' 'KR' 'MQ' 'BG' 'MA' 'ID' 'EC' 'BD' 'EE' 'DZ' 'SX' 'KE'
 'AL' 'RU' 'SM' 'XK' 'JM' '.PL' 'MD' 'OM' 'BF']

Unique levels for column 'star_rating_': [5 1 2 3 4]

Count for the Countries: 93

 Labeled Data Count: 627

 Unlabeled Data Count: 5095


In [92]:
df_reviews_filtered = df.dropna(subset=['text_reviews_']).loc[df['text_reviews_'] != '']

# Function to detect language
def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return 'unknown'

# Apply language detection function to the text reviews column
df_reviews_filtered['language'] = df_reviews_filtered['text_reviews_'].apply(detect_language)

# Count the number of text reviews that are not in English or are empty
non_english_count = df_reviews_filtered[(df_reviews_filtered['language'] != 'en')].shape[0]

print("Count for text reviews that are not in English or are empty:", non_english_count)


Count for text reviews that are not in English or are empty: 2923


**Create a Definition: Custom cleaning function**

In [93]:
translator = Translator()

# Translate non-English texts to English
def translate_to_english(text):
    try:
        # Check if the language of the text is not English
        if detect(text) != 'en':
            translated_text = translator.translate(text, dest='en').text
            return translated_text
        else:
            return text  # Return original text if already in English
    except:
        return text  # Return original text if translation fails

# Apply translation to non-English texts
df['formatted_reviews'] = df['text_reviews_'].apply(translate_to_english)

# Print the DataFrame to verify the translated texts
print(df['formatted_reviews'])

0       Fast shipping! All cloths was finally fits wel...
1       Just wanted to say how delighted I was with th...
2       My order took 6 days with a snow day and a wee...
3       Wouldnt give them no stars. I ordered a coat 4...
4       Parcel never arrived. Chasing for a refund for...
                              ...                        
5717    IÃÂ¯ÃÂ¾ÃÆÃÂ£ÃÆÃÂ£ÃÂ¯ÃÂ¾ÃÆÃÂ¯ÃÂ½Ã...
5718    I wanted to change the delivery location of my...
5719    Waiting since 24/14/2022. I believe the sent m...
5720    Who do you complain to about late deliveries f...
5721    Placed an order on the 24th November. H_ alert...
Name: formatted_reviews, Length: 5722, dtype: object


In [94]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = text.lower()  # Lowercase text
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove words with 1 or 2 letters
    text = re.sub(r'[^a-z\s]', '', text)  # Keep text with letters and spaces

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

**Clean the Reviews**

In [95]:
df['cleaned_reviews'] = df['formatted_reviews'].apply(clean_text)
print("Total cleaned reviews = ", len(df['cleaned_reviews']), "\n\n", df["cleaned_reviews"])

Total cleaned reviews =  5722 

 0          fast shipping cloth finally fit well satisfied
1       wanted say delighted friendly helpful staff me...
2       order took day snow day weekend happy sweater ...
3       wouldnt give star ordered coat week ago receiv...
4       parcel never arrived chasing refund substantia...
                              ...                        
5717    given star could known evri hermes courier wou...
5718    wanted change delivery location item within ho...
5719    waiting since believe sent order wrong country...
5720    complain late delivery horrendous compnay revi...
5721    placed order th november alert order could tak...
Name: cleaned_reviews, Length: 5722, dtype: object


**Perform Self-Learning**

Split data into labeled and unlabeled

In [96]:
# unlabeled_data = df[df['emotions_'] == 'NaN'][['cleaned_reviews']]

unlabeled_data = df[pd.isnull(df['emotions_'])][['cleaned_reviews']]

print(len(unlabeled_data))

unlabeled_data['emotions_'] = -1

unlabeled_data

5095


,cleaned_reviews,emotions_
0,fast shipping cloth finally fit well satisfied,-1
1,wanted say delighted friendly helpful staff me...,-1
4,parcel never arrived chasing refund substantia...,-1
5,avoid ordering online ordered day ago evri pac...,-1
6,awful service returning item booked collection...,-1
...,...,...
5717,given star could known evri hermes courier wou...,-1
5718,wanted change delivery location item within ho...,-1
5719,waiting since believe sent order wrong country...,-1
5720,complain late delivery horrendous compnay revi...,-1


**Extract X and y from labeled_data**

In [97]:
# Define labeled data as data where "Sentiment" is not missing
labeled_data = df[df['emotions_'].notnull() & (df['emotions_'] != 'NaN')]

print("Labeled data count:", len(labeled_data))

# Extract labels from labeled_data
y_labeled = labeled_data['emotions_']
y_unlabeled = unlabeled_data['emotions_']
X_labeled = labeled_data['cleaned_reviews']
X_unlabeled = unlabeled_data['cleaned_reviews']

# print("y_labeled ",y_labeled )
# print("y_unlabeled",y_unlabeled)
# print("X_unlabeled",X_unlabeled)
# print("X_labeled",X_labeled)


Labeled data count: 627


**Pipelines**

In [98]:
from sklearn.svm import SVC  # Import Support Vector Classification
from sklearn.model_selection import GridSearchCV

# Parameters
svm_params = dict(decision_function_shape='ovo', C=1.0, kernel='linear', gamma='auto', probability=True, random_state=42)
# hyper_parameters = {
#                     'kernel': ['linear'],
#                     'C': [0.01, 0.1],
#                     'gamma': [0.01, 0.1]}

vectorizer_params = dict(ngram_range=(1, 2), min_df=1, max_df=0.8)

# Supervised Pipeline with SVM
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        # ("clf", GridSearchCV(SVC(**svm_params), param_grid=hyper_parameters, cv=5))
        ("clf", SVC(**svm_params)),
    ]
)
# SelfTraining Pipeline
st_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SelfTrainingClassifier(SVC(**svm_params), verbose=True)),
        # ("clf", SelfTrainingClassifier(GridSearchCV(SVC(**svm_params), param_grid=hyper_parameters, cv=3), verbose=True)),

    ]
)

**Define a function for a classification report**

In [99]:
def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test, predict_df_unlabeled=0):
    print("\n\033[1mNumber of training samples:\033[0m", len(X_train))
    print("\n\033[1mUnlabeled samples in training set:\033[0m", sum(1 for x in y_train if x == -1)) #if x == NULL

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    #print("y Train", y_train)

    print("\ny Test",y_test)
    print("\ny Predict",y_pred)

    print(
        "\n\033[1mMicro-averaged F1 score on test set:\033[0m %0.3f"
        % f1_score(y_test, y_pred, average="micro")
    )
    print("\n\033[1m\nConfusion Matrix:\033[0m", confusion_matrix(y_test,y_pred))
    print("\n\033[1mClassification Report:\033[0m", classification_report(y_test, y_pred,zero_division=1))

    if predict_df_unlabeled:
      # Predict emotions in dataframe for text data without labels and saving to column ypred_emotions
      ypred_emotions = clf.predict(df[df['emotions_'].isnull()]['cleaned_reviews'])

      # Assign the predicted emotions to a new column in the DataFrame
      df.loc[df['emotions_'].isnull(), 'ypred_emotions'] = ypred_emotions
      print(len(df['ypred_emotions']))

      # Reverse the label_mapping dictionary
      reverse_label_mapping = {v: k for k, v in label_mapping.items()}

      # Convert predicted labels to emotions
      df['ypred_emotions'] = df['ypred_emotions'].map(reverse_label_mapping)

      print(df['ypred_emotions'])

**Split the data**

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, stratify=y_labeled, random_state=42)

print(len(X_train), len(y_train), len(X_test), len(y_test))

501 501 126 126


**Supervised SVM Classifier on the labeled data**

In [101]:
print("Supervised SVM Classifier on the labeled data:")
y_pred = eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

Supervised SVM Classifier on the labeled data:

Number of training samples: 501

Unlabeled samples in training set: 0

y Test 304     surprise
57           joy
912     surprise
3367    surprise
2820     neutral
          ...   
703        anger
2687     neutral
3339    surprise
1674     sadness
4890    surprise
Name: emotions_, Length: 126, dtype: object

y Predict ['surprise' 'joy' 'surprise' 'surprise' 'neutral' 'joy' 'surprise'
 'surprise' 'disgust' 'surprise' 'sadness' 'surprise' 'joy' 'joy' 'joy'
 'disgust' 'anger' 'sadness' 'neutral' 'sadness' 'sadness' 'surprise'
 'joy' 'neutral' 'sadness' 'disgust' 'sadness' 'disgust' 'surprise'
 'anger' 'disgust' 'neutral' 'disgust' 'neutral' 'sadness' 'sadness'
 'sadness' 'surprise' 'disgust' 'surprise' 'sadness' 'sadness' 'disgust'
 'fear' 'sadness' 'surprise' 'disgust' 'neutral' 'sadness' 'surprise'
 'sadness' 'surprise' 'fear' 'sadness' 'fear' 'neutral' 'disgust'
 'neutral' 'surprise' 'joy' 'fear' 'surprise' 'neutral' 'joy' 'sadness'
 'sur

**Self Training Classifier on the labeled data**

In [102]:
print("Self Training Classifier on the labeled data:")
eval_and_print_metrics(st_pipeline, X_train, y_train, X_test, y_test)

Self Training Classifier on the labeled data:

Number of training samples: 501

Unlabeled samples in training set: 0


/usr/local/lib/python3.10/dist-packages/sklearn/semi_supervised/_self_training.py:212: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)



y Test 304     surprise
57           joy
912     surprise
3367    surprise
2820     neutral
          ...   
703        anger
2687     neutral
3339    surprise
1674     sadness
4890    surprise
Name: emotions_, Length: 126, dtype: object

y Predict ['surprise' 'joy' 'surprise' 'surprise' 'neutral' 'joy' 'surprise'
 'surprise' 'disgust' 'surprise' 'sadness' 'surprise' 'joy' 'joy' 'joy'
 'disgust' 'anger' 'sadness' 'neutral' 'sadness' 'sadness' 'surprise'
 'joy' 'neutral' 'sadness' 'disgust' 'sadness' 'disgust' 'surprise'
 'anger' 'disgust' 'neutral' 'disgust' 'neutral' 'sadness' 'sadness'
 'sadness' 'surprise' 'disgust' 'surprise' 'sadness' 'sadness' 'disgust'
 'fear' 'sadness' 'surprise' 'disgust' 'neutral' 'sadness' 'surprise'
 'sadness' 'surprise' 'fear' 'sadness' 'fear' 'neutral' 'disgust'
 'neutral' 'surprise' 'joy' 'fear' 'surprise' 'neutral' 'joy' 'sadness'
 'surprise' 'joy' 'neutral' 'disgust' 'surprise' 'surprise' 'surprise'
 'surprise' 'anger' 'neutral' 'fear' 'surprise' 'dis

**Self Training Classifier on the labeled and unlabeled data**

**Manage Labeled and Unlabeled Data**

In [103]:
test_indices = X_test.index
#print("TEST INDICES",test_indices)

# Exclude test data from X_labeled and y_labeled based on the identified indices
X_labeled_filtered = X_labeled.drop(index=test_indices, errors='ignore')
y_labeled_filtered = y_labeled.drop(index=test_indices, errors='ignore')

# Concatenate the filtered labeled data with the unlabeled data
X=X_combined = pd.concat([X_labeled_filtered, X_unlabeled])
y=y_combined = pd.concat([y_labeled_filtered, y_unlabeled])


**Mapping Labels**

In [104]:
# Define the mapping for labels
# label_mapping = {'Positive': 1, 'Negative': 0, -1:-1 }
label_mapping = {
    'anger': 1,
    'joy': 2,
    'sadness': 3,
    'fear': 4,
    'disgust': 5,
    'surprise': 6,
    'neutral': 7,
    -1:-1
}
# Apply the mapping to labels
y  = [label_mapping[label] for label in y]
#print(y)
y_test  = [label_mapping[label] for label in y_test]
#print(y_test)

In [105]:
print("Self Training Classifier on the labeled and unlabeled data:")
y_pred = eval_and_print_metrics(st_pipeline, X, y, X_test, y_test, predict_df_unlabeled=1)

Self Training Classifier on the labeled and unlabeled data:

Number of training samples: 5596

Unlabeled samples in training set: 5095
End of iteration 1, added 170 new labels.
End of iteration 2, added 346 new labels.
End of iteration 3, added 779 new labels.
End of iteration 4, added 444 new labels.
End of iteration 5, added 165 new labels.
End of iteration 6, added 235 new labels.
End of iteration 7, added 174 new labels.
End of iteration 8, added 170 new labels.
End of iteration 9, added 178 new labels.
End of iteration 10, added 188 new labels.

y Test [6, 2, 6, 6, 7, 3, 6, 7, 5, 6, 1, 1, 2, 2, 2, 6, 5, 1, 7, 7, 6, 7, 2, 3, 2, 5, 4, 5, 6, 1, 5, 7, 3, 7, 4, 2, 5, 6, 4, 5, 4, 3, 7, 6, 3, 7, 7, 7, 5, 6, 3, 2, 3, 2, 4, 7, 5, 7, 6, 2, 4, 7, 3, 1, 3, 6, 5, 3, 6, 2, 4, 5, 4, 1, 1, 4, 7, 5, 3, 4, 6, 5, 3, 2, 7, 6, 2, 7, 2, 3, 5, 6, 2, 3, 2, 6, 2, 3, 2, 4, 3, 5, 3, 2, 1, 5, 1, 3, 4, 4, 7, 4, 2, 1, 4, 3, 6, 6, 7, 4, 6, 1, 7, 6, 3, 6]

y Predict [5 2 2 7 7 2 2 7 5 6 2 2 2 2 2 5 1 7 7 2 5 7 2

In [ ]:
# Function to combine values from emotions_ and ypred_emotions columns
def combine_emotions(row):
    if pd.notnull(row['emotions_']):
        return row['emotions_']
    else:
        return row['ypred_emotions']

# Apply the function to create the emotions_combined column
df['emotions_combined'] = df.apply(combine_emotions, axis=1)

# Specify the file path where you want to save the CSV file
file_path = "output.csv"

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)